## Import

In [1]:
import os
import random

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb

In [3]:
from neumeta.models import create_model_cifar10 as create_model
from neumeta.utils import (
    parse_args, print_omegaconf,
    load_checkpoint, save_checkpoint,
    set_seed,
    get_cifar10, 
    sample_coordinates, sample_subset, shuffle_coordinates_all,
    get_hypernetwork, get_optimizer, 
    sample_weights,
    weighted_regression_loss, validate_single, AverageMeter, EMA,
    sample_single_model, sample_merge_model,
)

## Functions

### 1 Find maximum dimension of the model

In [4]:
def find_max_dim(model_cls):
    """Find maximum dimension of the model"""
    # Get the learnable parameters of the model
    checkpoint = model_cls.learnable_parameter 

    # Set the maximum value to the length of the checkpoint
    max_value = len(checkpoint)

    # Iterate over the new model's weight
    for i, (k, tensor) in enumerate(checkpoint.items()):
        # Handle 2D tensors (e.g., weight matrices) 
        if len(tensor.shape) == 4:
            coords = [tensor.shape[0], tensor.shape[1]]
            max_value = max(max_value, max(coords))
        # Handle 1D tensors (e.g., biases)
        elif len(tensor.shape) == 1:
            max_value = max(max_value, tensor.shape[0])
    
    return max_value

### 2 Initialize wandb

In [5]:
def initialize_wandb(config):
    import time
    """
    Initializes Weights and Biases (wandb) with the given configuration.
    
    Args:
        configuration (dict): Configuration parameters for the run.
    """
    # Name the run using current time and configuration name
    run_name = f"{time.strftime('%Y%m%d%H%M%S')}-{config.experiment.name}"
    
    wandb.init(project="ninr-trial", name=run_name, config=dict(config), group='cifar10')

### 3 Initialize model dictionary

In [ ]:
def init_model_dict(args, device):
    """
    Initializes a dictionary of models for each dimension in the given range, along with ground truth models for the starting dimension.

    Args:
        args: An object containing the arguments for initializing the models.

    Returns:
        dim_dict: A dictionary containing the models for each dimension, along with their corresponding coordinates, keys, indices, size, and ground truth models.
        gt_model_dict: A dictionary containing the ground truth models for the starting dimension.
    """
    dim_dict = {}
    gt_model_dict = {}
    
    # Create a model for each dimension in dimensions range
    for dim in args.dimensions.range:
        model_cls = create_model(args.model.type,
                                 hidden_dim=dim,
                                 path=args.model.pretrained_path,
                                 smooth=args.model.smooth).to(device)
        # Sample the coordinates, keys, indices, and the size for the model
        coords_tensor, keys_list, indices_list, size_list = sample_coordinates(model_cls)
        # Add the model, coordinates, keys, indices, size, and key mask to the dictionary
        dim_dict[f"{dim}"] = (model_cls, coords_tensor, keys_list, indices_list, size_list, None)

        # Print to makes line better
        print('\n')
        
        # If the dimension is the starting dimension (the dimension of pretrained_model), add the ground truth model to the dictionary
        if dim == args.dimensions.start:
            print(f"Loading model for dim {dim}")
            model_trained = create_model(args.model.type, 
                                         hidden_dim=dim, 
                                         path=args.model.pretrained_path, 
                                         smooth=args.model.smooth).to(device)
            model_trained.eval()
            gt_model_dict[f'{dim}'] = model_trained

    
    return dim_dict, gt_model_dict

### 4 Training function for target model of a random dimension

In [7]:
# Function to train the model for one epoch
def train_one_epoch(model, train_loader, optimizer, criterion, dim_dict, gt_model_dict, epoch_idx, ema=None, args=None, device='cpu'):
    # Set the model to training mode
    model.train()
    total_loss = 0.0

    # Initialize AverageMeter objects to track the losses
    losses = AverageMeter()
    cls_losses = AverageMeter()
    reg_losses = AverageMeter()
    reconstruct_losses = AverageMeter()

    # Iterate over the training data
    for batch_idx, (x, target) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()

        # Preprocess input
        # ------------------------------------------------------------------------------------------------------
        # Move the data to the device
        x, target = x.to(device), target.to(device)
        # Choose a random hidden dimension
        hidden_dim = random.choice(args.dimensions.range)
        # Get the model class, coordinates, keys, indices, size, and key mask for the chosen dimension
        model_cls, coords_tensor, keys_list, indices_list, size_list, key_mask = dim_dict[f"{hidden_dim}"]
        # Sample a subset the input tensor of the coordinates, keys, indices, size, and selected keys
        coords_tensor, keys_list, indices_list, size_list, selected_keys = sample_subset(coords_tensor,
                                                                                         keys_list,
                                                                                         indices_list,
                                                                                         size_list,
                                                                                         key_mask,
                                                                                         ratio=args.ratio)
        # Add noise to the coordinates if specified
        if args.training.coordinate_noise > 0.0:
            coords_tensor = coords_tensor + (torch.rand_like(coords_tensor) - 0.5) * args.training.coordinate_noise


        # Main task of hypernetwork and target network
        # ------------------------------------------------------------------------------------------------------
        # Sample the weights for the target model using hypernetwork
        model_cls, reconstructed_weights = sample_weights(model, model_cls,
                                                          coords_tensor, keys_list, indices_list, size_list, key_mask, selected_keys,
                                                          device=device, NORM=args.dimensions.norm)
        # Forward pass
        predict = model_cls(x)


        # Compute losses
        # ------------------------------------------------------------------------------------------------------
        # Compute classification loss
        cls_loss = criterion(predict, target) 
        # Compute regularization loss
        reg_loss = sum([torch.norm(w, p=2) for w in reconstructed_weights])
        # Compute reconstruction loss if ground truth model is available
        if f"{hidden_dim}" in gt_model_dict:
            gt_model = gt_model_dict[f"{hidden_dim}"]
            gt_selected_weights = [
                w for k, w in gt_model.learnable_parameter.items() if k in selected_keys]

            reconstruct_loss = weighted_regression_loss(
                reconstructed_weights, gt_selected_weights)
        else:
            reconstruct_loss = torch.tensor(0.0)
        # Compute the total loss
        loss = args.hyper_model.loss_weight.ce_weight * cls_loss + args.hyper_model.loss_weight.reg_weight * \
            reg_loss + args.hyper_model.loss_weight.recon_weight * reconstruct_loss


        # Compute gradients and update weights
        # ------------------------------------------------------------------------------------------------------
        # Zero the gradients of the updated weights
        for updated_weight in model_cls.parameters():
            updated_weight.grad = None

        # Compute the gradients of the reconstructed weights
        loss.backward(retain_graph=True)
        torch.autograd.backward(reconstructed_weights, [
                                w.grad for k, w in model_cls.named_parameters() if k in selected_keys])
        
        # Clip the gradients if specified
        if args.training.get('clip_grad', 0.0) > 0:
            torch.nn.utils.clip_grad_value_(
                model.parameters(), args.training.clip_grad)
            
        # Update the weights
        optimizer.step()

        # Update the EMA if specified
        if ema:
            ema.update()  # Update the EMA after each training step
        total_loss += loss.item()

        # Update the AverageMeter objects
        losses.update(loss.item())
        cls_losses.update(cls_loss.item())
        reg_losses.update(reg_loss.item())
        reconstruct_losses.update(reconstruct_loss.item())


        # Log (or plot) losses
        # ------------------------------------------------------------------------------------------------------
        # Log the losses and learning rate to wandb
        if batch_idx % args.experiment.log_interval == 0:
            wandb.log({
                "Loss": losses.avg,
                "Cls Loss": cls_losses.avg,
                "Reg Loss": reg_losses.avg,
                "Reconstruct Loss": reconstruct_losses.avg,
                "Learning rate": optimizer.param_groups[0]['lr']
            }, step=batch_idx + epoch_idx * len(train_loader))
            # Print the losses and learning rate
            print(
                f"Iteration {batch_idx}: Loss = {losses.avg:.4f}, Reg Loss = {reg_losses.avg:.4f}, Reconstruct Loss = {reconstruct_losses.avg:.4f}, Cls Loss = {cls_losses.avg:.4f}, Learning rate = {optimizer.param_groups[0]['lr']:.4e}")
    
    # Returns the training loss, structure of network in each dimension, and the original structure of pretrained network
    return losses.avg, dim_dict, gt_model_dict

## Main

### 0 Set device to GPU

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 1 Parsing arguments for inputs

In [9]:
CONFIG_PATH = 'neumeta/config/base_config.yaml'
RATIO = '1.0'

In [10]:
argv_train = ['--config', CONFIG_PATH, '--ratio', RATIO]
argv_test = ['--config', CONFIG_PATH, '--test']

In [11]:
args = parse_args(argv_train)  # Parse arguments
print_omegaconf(args)  # Print arguments

+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|                 Key                  |                                                                Value                                                                 |
+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|           experiment.name            |                                                 ninr_resnet20_cifar10_32-64-4layer-5                                                 |
|         experiment.recononly         |                                                                  0                                                                   |
|        experiment.num_epochs         |                                                                  30            

### 2 Get training and validation data (in dataloader format)

In [12]:
train_loader, val_loader = get_cifar10(args.training.batch_size, strong_transform=args.training.get('strong_aug', None))

### 3 Create target model

#### 3.0 Create the model

In [13]:
model = create_model(args.model.type,
                     hidden_dim=args.dimensions.start,
                     path=args.model.pretrained_path,
                     smooth=args.model.smooth).to(device)

Replace the last 2 block of layer3 with new block with hidden dim 64
Loading pretrained weights for resnet20


#### 3.1 Print the structure and shape of the model

In [14]:
model

CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (2): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(


In [15]:
for i, (k, tensor) in enumerate(model.learnable_parameter.items()):
    print(k, tensor.shape)

layer3.2.conv1.weight torch.Size([64, 64, 3, 3])
layer3.2.conv1.bias torch.Size([64])
layer3.2.conv2.weight torch.Size([64, 64, 3, 3])
layer3.2.conv2.bias torch.Size([64])


#### 3.2 The maximum dimension of the target model

In [16]:
# Print the maximum dimension of the model
print(f'Maximum DIM: {find_max_dim(model)}')

Maximum DIM: 64


#### 3.3 Validate the accuracy of pretrained network

In [17]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(model, val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 157/157 [00:02<00:00, 70.03it/s]

Initial Permutated model Validation Loss: 0.2825, Validation Accuracy: 92.60%


In [18]:
# Get the learnable parameters of the model
checkpoint = model.learnable_parameter
# Get the number of parameters
number_param = len(checkpoint)

In [19]:
# Print the keys of the parameters and the number of parameters
print(f"Parameters keys: {model.keys}")
print(f"Number of parameters to be learned: {number_param}")

Parameters keys: ['layer3.2.conv1.weight', 'layer3.2.conv1.bias', 'layer3.2.conv2.weight', 'layer3.2.conv2.bias']
Number of parameters to be learned: 4


### 4 Create the hypernetwork

#### 4.0 Create the model

In [20]:
# Get the hypermodel
hyper_model = get_hypernetwork(args, number_param)

Hyper model type: mlp
num_freqs:  16 <class 'int'>


#### 4.1 Print model structure

In [21]:
hyper_model

NeRF_MLP_Compose(
  (positional_encoding): PositionalEncoding()
  (model): ModuleList(
    (0-3): 4 x NeRF_MLP_Residual_Scaled(
      (initial_layer): Linear(in_features=198, out_features=256, bias=True)
      (residual_blocks): ModuleList(
        (0-2): 3 x Linear(in_features=256, out_features=256, bias=True)
      )
      (scalars): ParameterList(
          (0): Parameter containing: [torch.float32 of size  (cuda:0)]
          (1): Parameter containing: [torch.float32 of size  (cuda:0)]
          (2): Parameter containing: [torch.float32 of size  (cuda:0)]
      )
      (act): ReLU(inplace=True)
      (output_layer): Linear(in_features=256, out_features=9, bias=True)
    )
  )
)

#### 4.2 Initialize EMA to track only a smooth version of the model weight

In [22]:
# Initialize the EMA
ema = EMA(hyper_model, decay=args.hyper_model.ema_decay)

### 5 Get Loss function, Optimizer, and Scheduler

In [23]:
criterion, val_criterion, optimizer, scheduler = get_optimizer(args, hyper_model)

In [24]:
print(f'Criterion: {criterion}\nVal_criterion: {val_criterion}\nOptimizer: {optimizer}\nScheduler: {scheduler}')

Criterion: CrossEntropyLoss()
Val_criterion: CrossEntropyLoss()
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)
Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001467EAD4E10>


### 6 Training loop

#### 6.1 Initialize training parameters

In [25]:
# Initialize the starting epoch and best accuracy
start_epoch = 0
best_acc = 0.0

#### 6.2 Directory to save the model

In [26]:
# Create the directory to save the model
os.makedirs(args.training.save_model_path, exist_ok=True)

#### 6.3 Resume training loop

In [27]:
if args.resume_from:
        print(f"Resuming from checkpoint: {args.resume_from}")
        checkpoint_info = load_checkpoint(args.resume_from, hyper_model, optimizer, ema)
        start_epoch = checkpoint_info['epoch']
        best_acc = checkpoint_info['best_acc']
        print(f"Resuming from epoch: {start_epoch}, best accuracy: {best_acc*100:.2f}%")
        # Note: If there are more elements to retrieve, do so here.

#### 6.4 Initialize wandb for plotting

In [28]:
# Initialize wandb
initialize_wandb(args)

wandb: Currently logged in as: efradosuryadi (efradosuryadi-universitas-indonesia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


#### 6.5 Initialize model dictionary for each dimension and shuffle it

In [29]:
# Initialize model dictionary
dim_dict, gt_model_dict = init_model_dict(args, device)

Replace the last 2 block of layer3 with new block with hidden dim 32
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 33
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 34
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 35
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 36
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 37
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 38
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 39
Loading pretrained weights for resnet20
Replace the last 2 block of layer3 with new block with hidden dim 40
Loading pretrained weights for resnet20
Replace the last 2 

In [30]:
gt_model_dict

{'64': CifarResNet(
   (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (bn1): Identity()
   (relu): ReLU(inplace=True)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
     )
     (1): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
     )
     (2): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
   

In [31]:
dim_dict

{'32': (CifarResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): Identity()
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (2): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [32]:
dim_dict = shuffle_coordinates_all(dim_dict)
dim_dict

{'32': (CifarResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): Identity()
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (2): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

#### 6.6 Hypernetwork training loop

In [33]:
args.experiment.num_epochs

30

In [34]:
# Iterate over the epochs
for epoch in range(start_epoch, args.experiment.num_epochs):
    # Train the hypernetwork to generate a model with random dimension for one epoch
    train_loss, dim_dict, gt_model_dict = train_one_epoch(hyper_model, train_loader, optimizer, criterion, 
                                                          dim_dict, gt_model_dict, epoch_idx=epoch, ema=ema, 
                                                          args=args, device=device)
    # Step the scheduler
    scheduler.step()

    # Print the training loss and learning rate
    print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Training Loss: {train_loss:.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    # If it's time to evaluate the model
    if (epoch + 1) % args.experiment.eval_interval == 0:
        # Apply EMA if it is specified
        if ema:
            ema.apply()  # Save the weights of original model created before training_loop
        
        # Sample the merged model (create model of same structure before training loop by using the hypernetwork)
        # And then test the performance of the hypernetwork by seeing how good it is in generating the weights
        model = sample_merge_model(hyper_model, model, args) 
        # Validate the merged model
        val_loss, acc = validate_single(model, val_loader, val_criterion, args=args)

        # If EMA is specified, restore the original weights
        if ema:
            ema.restore()  # Restore the original weights to the weights of the pretrained networks

        # Log the validation loss and accuracy to wandb
        wandb.log({
            "Validation Loss": val_loss,
            "Validation Accuracy": acc
        })
        # Print the validation loss and accuracy
        print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
        print('\n\n')

        # Save the checkpoint if the accuracy is better than the previous best
        if acc > best_acc:
            best_acc = acc
            save_checkpoint(f"{args.training.save_model_path}/cifar10_nerf_best.pth",hyper_model,optimizer,ema,epoch,best_acc)
            print(f"Checkpoint saved at epoch {epoch} with accuracy: {best_acc*100:.2f}%")


Iteration 0: Loss = 1.8819, Reg Loss = 1.9512, Reconstruct Loss = 0.0000, Cls Loss = 1.8817, Learning rate = 1.0000e-03
Iteration 25: Loss = 1.8710, Reg Loss = 9.3402, Reconstruct Loss = 0.0064, Cls Loss = 1.8636, Learning rate = 1.0000e-03
Iteration 50: Loss = 1.8601, Reg Loss = 7.4991, Reconstruct Loss = 0.0033, Cls Loss = 1.8561, Learning rate = 1.0000e-03
Iteration 75: Loss = 1.8558, Reg Loss = 6.7918, Reconstruct Loss = 0.0022, Cls Loss = 1.8529, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8533, Reg Loss = 6.1977, Reconstruct Loss = 0.0018, Cls Loss = 1.8509, Learning rate = 1.0000e-03
Iteration 125: Loss = 1.8518, Reg Loss = 7.2052, Reconstruct Loss = 0.0015, Cls Loss = 1.8496, Learning rate = 1.0000e-03
Iteration 150: Loss = 1.8514, Reg Loss = 7.9440, Reconstruct Loss = 0.0012, Cls Loss = 1.8494, Learning rate = 1.0000e-03
Iteration 175: Loss = 1.8502, Reg Loss = 7.8039, Reconstruct Loss = 0.0011, Cls Loss = 1.8484, Learning rate = 1.0000e-03
Iteration 200: Loss = 1.8493,

100%|██████████| 157/157 [00:02<00:00, 64.79it/s]


Epoch [1/30], Validation Loss: 1.8046, Validation Accuracy: 78.96%



Checkpoint saved at epoch 0 with accuracy: 78.96%
Iteration 0: Loss = 1.8215, Reg Loss = 25.0542, Reconstruct Loss = 0.0000, Cls Loss = 1.8190, Learning rate = 1.0000e-03
Iteration 25: Loss = 1.8388, Reg Loss = 25.6970, Reconstruct Loss = 0.0000, Cls Loss = 1.8362, Learning rate = 1.0000e-03
Iteration 50: Loss = 1.8355, Reg Loss = 24.3616, Reconstruct Loss = 0.0007, Cls Loss = 1.8324, Learning rate = 1.0000e-03
Iteration 75: Loss = 1.8346, Reg Loss = 22.0040, Reconstruct Loss = 0.0009, Cls Loss = 1.8315, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8362, Reg Loss = 19.9200, Reconstruct Loss = 0.0007, Cls Loss = 1.8335, Learning rate = 1.0000e-03
Iteration 125: Loss = 1.8382, Reg Loss = 17.2406, Reconstruct Loss = 0.0008, Cls Loss = 1.8357, Learning rate = 1.0000e-03
Iteration 150: Loss = 1.8373, Reg Loss = 15.1631, Reconstruct Loss = 0.0007, Cls Loss = 1.8351, Learning rate = 1.0000e-03
Iteration 175: Loss = 1.

100%|██████████| 157/157 [00:03<00:00, 43.54it/s]


Epoch [2/30], Validation Loss: 1.8042, Validation Accuracy: 79.39%



Checkpoint saved at epoch 1 with accuracy: 79.39%
Iteration 0: Loss = 1.8441, Reg Loss = 16.2423, Reconstruct Loss = 0.0000, Cls Loss = 1.8424, Learning rate = 1.0000e-03
Iteration 25: Loss = 1.8660, Reg Loss = 33.9974, Reconstruct Loss = 0.0012, Cls Loss = 1.8614, Learning rate = 1.0000e-03
Iteration 50: Loss = 1.8582, Reg Loss = 58.0114, Reconstruct Loss = 0.0016, Cls Loss = 1.8509, Learning rate = 1.0000e-03
Iteration 75: Loss = 1.8523, Reg Loss = 65.6450, Reconstruct Loss = 0.0011, Cls Loss = 1.8446, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8499, Reg Loss = 66.9480, Reconstruct Loss = 0.0012, Cls Loss = 1.8421, Learning rate = 1.0000e-03
Iteration 125: Loss = 1.8499, Reg Loss = 64.4183, Reconstruct Loss = 0.0012, Cls Loss = 1.8422, Learning rate = 1.0000e-03
Iteration 150: Loss = 1.8489, Reg Loss = 57.9702, Reconstruct Loss = 0.0012, Cls Loss = 1.8419, Learning rate = 1.0000e-03
Iteration 175: Loss = 1.

...

Epoch [28/30], Validation Loss: 1.8076, Validation Accuracy: 75.87%



Iteration 0: Loss = 1.9651, Reg Loss = 1116.4069, Reconstruct Loss = 0.0000, Cls Loss = 1.8534, Learning rate = 1.0000e-03
Iteration 25: Loss = 1.9697, Reg Loss = 1021.4845, Reconstruct Loss = 0.0369, Cls Loss = 1.8306, Learning rate = 1.0000e-03
Iteration 50: Loss = 1.9748, Reg Loss = 1019.4860, Reconstruct Loss = 0.0368, Cls Loss = 1.8360, Learning rate = 1.0000e-03
Iteration 75: Loss = 1.9623, Reg Loss = 1004.6599, Reconstruct Loss = 0.0247, Cls Loss = 1.8372, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.9570, Reg Loss = 1000.5599, Reconstruct Loss = 0.0186, Cls Loss = 1.8384, Learning rate = 1.0000e-03
Iteration 125: Loss = 1.9596, Reg Loss = 996.3730, Reconstruct Loss = 0.0215, Cls Loss = 1.8384, Learning rate = 1.0000e-03
Iteration 150: Loss = 1.9554, Reg Loss = 982.6280, Reconstruct Loss = 0.0180, Cls Loss = 1.8392, Learning rate = 1.0000e-03
Iteration 175: Loss = 1.9564, Reg Loss = 975.0827, Reconstruc

100%|██████████| 157/157 [00:02<00:00, 70.37it/s]


Epoch [29/30], Validation Loss: 1.8076, Validation Accuracy: 75.86%



Iteration 0: Loss = 1.8926, Reg Loss = 227.6601, Reconstruct Loss = 0.0000, Cls Loss = 1.8698, Learning rate = 1.0000e-03
Iteration 25: Loss = 1.8633, Reg Loss = 226.8706, Reconstruct Loss = 0.0029, Cls Loss = 1.8377, Learning rate = 1.0000e-03
Iteration 50: Loss = 1.8660, Reg Loss = 206.5443, Reconstruct Loss = 0.0015, Cls Loss = 1.8438, Learning rate = 1.0000e-03
Iteration 75: Loss = 1.8646, Reg Loss = 188.2787, Reconstruct Loss = 0.0019, Cls Loss = 1.8439, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8638, Reg Loss = 172.3546, Reconstruct Loss = 0.0025, Cls Loss = 1.8441, Learning rate = 1.0000e-03
Iteration 125: Loss = 1.8620, Reg Loss = 156.0392, Reconstruct Loss = 0.0027, Cls Loss = 1.8437, Learning rate = 1.0000e-03
Iteration 150: Loss = 1.8593, Reg Loss = 139.2707, Reconstruct Loss = 0.0028, Cls Loss = 1.8426, Learning rate = 1.0000e-03
Iteration 175: Loss = 1.8756, Reg Loss = 160.7912, Reconstruct Los

100%|██████████| 157/157 [00:02<00:00, 71.20it/s]

Epoch [30/30], Validation Loss: 1.8076, Validation Accuracy: 75.86%





In [36]:
# End the wandb tracking
wandb.finish()

Cls Loss,▃▅▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▅▅▅▅▇▇█▅▅▅▅▅▅▅▅▅▅▅▅▅
Learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss,▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▂▂▂▂▃▁▂█▇▄▆▅▃▂▂▃▆▂▂▂▂▂▁▂▄
Reconstruct Loss,▁▁▁▁▁▁▁▁▁▂▂▁▂▂▁▁▁▁▂█▆▅▃▂▃▃▃▇▅▄▁▁▁▁▁▃▂▂▂▂
Reg Loss,▁▁▁▁▁▁▁▂▁▂▂▂▁▁▁▆▅▂▃▄▆▄▄█▆▄▃▂▂▄▂▃▂▆▃▇▇▆▅▅
Validation Accuracy,▆▇▆▇▇▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▃▂▂▂▁▁▁███████████████████████
Cls Loss,1.84399
Learning rate,0.001
Loss,1.94761
Reconstruct Loss,0.02328


### 7 Testing loop

In [39]:
for hidden_dim in range(16, 65):
    # Create a model for this given dimension
    model = create_model(args.model.type,
                         hidden_dim=hidden_dim,
                         path=args.model.pretrained_path,
                         smooth=args.model.smooth).to(device)
    
    # If EMA is specified, apply it
    if ema:
        print('Applying EMA')
        ema.apply()

    # Sample the merged model
    accumulated_model = sample_merge_model(hyper_model, model, args, K=100)

    # Validate the merged model
    val_loss, acc = validate_single(accumulated_model, val_loader, val_criterion, args=args)

    # If EMA is specified, restore the original weights after applying EMA
    if ema:
        ema.restore()  # Restore the original weights after applying 
        
    # Save the model
    save_name = os.path.join(args.training.save_model_path, f"cifar10_{accumulated_model.__class__.__name__}_dim{hidden_dim}_single.pth")
    torch.save(accumulated_model.state_dict(),save_name)

    # Print the results
    print(f"Test using model {args.model}: hidden_dim {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
    print('\n')

    # Define the directory and filename structure
    filename = f"cifar10_results_{args.experiment.name}.txt"
    filepath = os.path.join(args.training.save_model_path, filename)

    # Write the results. 'a' is used to append the results; a new file will be created if it doesn't exist.
    with open(filepath, "a") as file:
        file.write(f"Hidden_dim: {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%\n")


Replace the last 2 block of layer3 with new block with hidden dim 16
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 71.31it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'resnet20-12fca82f.th', 'smooth': False}: hidden_dim 16, Validation Loss: 1.8076, Validation Accuracy: 75.88%


Replace the last 2 block of layer3 with new block with hidden dim 17
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 70.78it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'resnet20-12fca82f.th', 'smooth': False}: hidden_dim 17, Validation Loss: 1.8076, Validation Accuracy: 75.88%


Replace the last 2 block of layer3 with new block with hidden dim 18
Loading pretrained weights for resnet20
Applying EMA


...

Test using model {'type': 'ResNet20', 'pretrained_path': 'resnet20-12fca82f.th', 'smooth': False}: hidden_dim 62, Validation Loss: 1.8076, Validation Accuracy: 75.88%


Replace the last 2 block of layer3 with new block with hidden dim 63
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 74.75it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'resnet20-12fca82f.th', 'smooth': False}: hidden_dim 63, Validation Loss: 1.8076, Validation Accuracy: 75.88%


Replace the last 2 block of layer3 with new block with hidden dim 64
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 77.24it/s]

Test using model {'type': 'ResNet20', 'pretrained_path': 'resnet20-12fca82f.th', 'smooth': False}: hidden_dim 64, Validation Loss: 1.8076, Validation Accuracy: 75.88%


